In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# read CSV file and rename columns
df = pd.read_csv('SenateData.csv')
df = df.rename(columns={'State':'State', 'Population 2010':'POP2010', 'Population 2019':'POP2019', 
                   'Senator Name':'SenatorName', 'Party':'Party', 'Share of Vote':'VoteShare'})

# insert column of 0s for 2021 Population Prediction
df.insert(3, 'POP2021', np.zeros(100))
df

,State,POP2010,POP2019,POP2021,SenatorName,Party,VoteShare
0,Alabama,4785437,4903185,0.0,Richard Shelby,R,0.641
1,Alabama,4785437,4903185,0.0,Tommy Tuberville,R,0.601
2,Alaska,713910,731545,0.0,Dan Sullivan,R,0.539
3,Alaska,713910,731545,0.0,Lisa Murkowski,R,0.501
4,Arizona,6407172,7278717,0.0,Mark Kelly,D,0.512
...,...,...,...,...,...,...,...
95,West Virginia,1854239,1792147,0.0,Shelley Moore-Capito,R,0.703
96,Wisconsin,5690475,5822434,0.0,Tammy Baldwin,D,0.554
97,Wisconsin,5690475,5822434,0.0,Ron Johnson,R,0.502
98,Wyoming,564487,578759,0.0,John Barrasso,R,0.671


In [3]:
# create list of states sorted in alphabetical order
states = df['State'].value_counts().index.tolist()
states.sort()

# for every state, calculate its annual growth rate
growth_rates = [0 for i in range (len(states))]
for index in range(len(growth_rates)):
    row = df[df['State'] == states[index]].iloc[0,:]
    growth_rates[index] = ((row['POP2019']/row['POP2010'])**(1/9))-1

# for every state, use its calculated growth rate 
# to predict its 2021 population
for index, row in df.iterrows():
    state_idx = states.index(row['State'])
    df.at[index,'POP2021'] = round(row['POP2019']*(1+growth_rates[state_idx])**2)
    
df

,State,POP2010,POP2019,POP2021,SenatorName,Party,VoteShare
0,Alabama,4785437,4903185,4929742.0,Richard Shelby,R,0.641
1,Alabama,4785437,4903185,4929742.0,Tommy Tuberville,R,0.601
2,Alaska,713910,731545,735523.0,Dan Sullivan,R,0.539
3,Alaska,713910,731545,735523.0,Lisa Murkowski,R,0.501
4,Arizona,6407172,7278717,7487958.0,Mark Kelly,D,0.512
...,...,...,...,...,...,...,...
95,West Virginia,1854239,1792147,1778634.0,Shelley Moore-Capito,R,0.703
96,Wisconsin,5690475,5822434,5852171.0,Tammy Baldwin,D,0.554
97,Wisconsin,5690475,5822434,5852171.0,Ron Johnson,R,0.502
98,Wyoming,564487,578759,581979.0,John Barrasso,R,0.671


In [4]:
# variables to track each party's "votes"

# for proportional approach
dem_votes_prop = 0
rep_votes_prop = 0

# for winner-take-all approach
dem_votes_wta = 0
rep_votes_wta = 0

# for every state, do the following
for state in states:
    
    # find the state's population
    state_pop = df[df['State'] == state]['POP2021'].iloc[0]
        
    # find each senator's party and vote share
    senator1_party = df[df['State'] == state].iloc[0,5]
    senator2_party = df[df['State'] == state].iloc[1,5]
    
    senator1_vs = df[df['State'] == state].iloc[0,6]
    senator2_vs = df[df['State'] == state].iloc[1,6]
    
    # divide the state's population
    # based on how it voted
    
    # if both senators are Democrats
    if senator1_party == 'D' and senator2_party == 'D':
        dem_votes = math.ceil((senator1_vs + senator2_vs)*state_pop/2)
        dem_votes_prop += dem_votes
        dem_votes_wta += state_pop
        
        rep_votes = state_pop - dem_votes
        rep_votes_prop += rep_votes
    
    # if both senators are Republicans
    elif senator1_party == 'R' and senator2_party == 'R':
        rep_votes = math.ceil((senator1_vs + senator2_vs)*state_pop/2)
        rep_votes_prop += rep_votes
        rep_votes_wta += state_pop
        
        dem_votes = state_pop - rep_votes
        dem_votes_prop += dem_votes
    
    # if there's one senator from each party
    else:
        dem_votes = math.ceil((senator1_vs + (1-senator2_vs))*state_pop/2)
        dem_votes_prop += dem_votes
        dem_votes_wta += state_pop/2
        
        rep_votes = state_pop - dem_votes
        rep_votes_prop += rep_votes
        rep_votes_wta += state_pop/2
        

In [5]:
# for both scenarios,
# subtract Republican population from Democratic population
print("Difference under winner-take-all: {:,.0f}".format(dem_votes_wta - rep_votes_wta))
print("Difference under proportional allocation: {:,.0f}".format(dem_votes_prop - rep_votes_prop))

Difference under winner-take-all: 40,945,687
Difference under proportional allocation: 15,485,513
